# Using Optimization Techniques to Generate Winning FanDuel Lineups
___
Until I build one from scratch I'm going to be testing out `pydfs-lineup-optimizer` to generate optimal lineups with projections that I already have. More information about `pydfs-lineup-optimizer` can be found [here](https://github.com/DimaKudosh/pydfs-lineup-optimizer/tree/master/pydfs_lineup_optimizer/sites/fanduel).

**Here is one quick example of generating lineups based on average points per game.**

In [37]:
import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, CSVLineupExporter

In [38]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('player-lists/Week-11/FanDuel-NFL-2018-11-18-29988-players-list.csv')
for lineup in optimizer.optimize(n=5, max_exposure=0.3):
    print(lineup.players)
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

[Ryan Fitzpatrick QB (TB), Alvin Kamara RB (NO), T.J. Yeldon RB (JAC), John Brown WR (BAL), Emmanuel Sanders WR (DEN), JuJu Smith-Schuster WR (PIT), Eric Ebron TE (IND), Washington Redskins D (WAS), Melvin Gordon III RB (LAC)]
148.415
60000.0
[Matt Ryan QB (ATL), Alvin Kamara RB (NO), T.J. Yeldon RB (JAC), John Brown WR (BAL), Emmanuel Sanders WR (DEN), DeSean Jackson WR (TB), Eric Ebron TE (IND), Washington Redskins D (WAS), Melvin Gordon III RB (LAC)]
148.348
59900.0
[Ryan Fitzpatrick QB (TB), Saquon Barkley RB (NYG), Joe Mixon RB (CIN), Dede Westbrook WR (JAC), Michael Thomas WR (NO), DeSean Jackson WR (TB), Jared Cook TE (OAK), Denver Broncos D (DEN), Marlon Mack RB (IND)]
136.537
59900.0
[Ben Roethlisberger QB (PIT), Saquon Barkley RB (NYG), Nyheim Hines RB (IND), Dede Westbrook WR (JAC), Michael Thomas WR (NO), Calvin Ridley WR (ATL), Zach Ertz TE (PHI), Denver Broncos D (DEN), Marlon Mack RB (IND)]
134.902
60000.0
[Ben Roethlisberger QB (PIT), Nyheim Hines RB (IND), Joe Mixon RB

In [39]:
problematic_names_dict = {'Melvin Gordon':'Melvin Gordon III', 'Odell Beckham':'Odell Beckham Jr.',\
                          'Will Fuller':'Will Fuller V', 'Marvin Jones':'Marvin Jones Jr.',\
                          'Willie Snead':'Willie Snead IV', 'Ted Ginn':'Ted Ginn Jr.', \
                          'Allen Robinson':'Allen Robinson II'}

## Now instead of just using the average points per game, we'll bring in our other projections that *should* be more accurate.

___
#### 1. Ballers

In [40]:
current_week = 12

qbs = pd.read_csv('projections/Week-{}/DFS - Fanduel QB.csv'.format(current_week))
rbs = pd.read_csv('projections/Week-{}/DFS - Fanduel RB.csv'.format(current_week))
wrs = pd.read_csv('projections/Week-{}/DFS - Fanduel WR.csv'.format(current_week))
tes = pd.read_csv('projections/Week-{}/DFS - Fanduel TE.csv'.format(current_week))
dst = pd.read_csv('projections/Week-{}/DFS - Fanduel DST.csv'.format(current_week))
full_team_names = list(dst.sort_values('Player')['Player'])

In [41]:
footballers = pd.concat([qbs,rbs,wrs,tes,dst])
footballers.rename(columns={"Player": "Nickname"}, inplace=True)
footballers.Nickname.replace(problematic_names_dict, inplace=True)

___
#### 2. Sportsline

In [42]:
sportsline = pd.read_csv('projections/Week-{}/projections.csv'.format(current_week))
sportsline.rename(columns={"PLAYER": "Nickname"}, inplace=True)
sportsline.Nickname.replace(problematic_names_dict, inplace=True)
abbv_team_names = list(sportsline.loc[sportsline.POS == 'DEF', :].sort_values('Nickname')['Nickname'])
defenses_dict = dict(zip(abbv_team_names, full_team_names))
sportsline['Nickname'].replace(defenses_dict, inplace = True)
sportsline.head(2)

,Nickname,POS,TEAM,OPP,FP,fd exp,dk exp,PPR,CONS%,FREB%,PASSYD,RUSHYD,RECYD
0,Andrew Luck,QB,IND,MIA,27.1,20.5,22.7,27.1,52%,38%,304.0,12.0,NaN
1,Tom Brady,QB,NE,@NYJ,24.4,20.0,23.0,24.4,43%,33%,309.0,3.0,NaN


___
#### 3. FantasyPros

In [43]:
fp_qb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_QB.csv'.format(current_week))
fp_rb = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_RB.csv'.format(current_week))
fp_wr = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_WR.csv'.format(current_week))
fp_te = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_TE.csv'.format(current_week))
fp_dst = pd.read_csv('projections/Week-{}/FantasyPros_Fantasy_Football_Projections_DST.csv'.format(current_week))

In [44]:
for df in [fp_qb, fp_rb, fp_wr, fp_te, fp_dst]:
    df.drop([col for col in df.columns if col not in ['Player', 'FPTS']], axis=1, inplace=True)

In [45]:
fantasy_pros = pd.concat([fp_qb, fp_rb, fp_wr, fp_te, fp_dst])
fantasy_pros.rename(columns={"Player": "Nickname"}, inplace=True)
fantasy_pros.Nickname.replace(problematic_names_dict, inplace=True)

___
#### Averaging the projections:

In [46]:
total_projections = pd.merge(footballers, sportsline, on="Nickname")
total_projections = pd.merge(total_projections, fantasy_pros, on="Nickname")
total_projections['Salary'] = total_projections.Salary.apply(lambda x: int(x.replace(',', '')))
total_projections['Pts'] = (total_projections.Pts + total_projections.FP + total_projections.FPTS) / 3
total_projections['Pts Per 1k'] = total_projections['Pts'] / (total_projections.Salary / 1000)
total_projections.drop(labels=list(total_projections.columns[5:]), axis=1, inplace=True)
total_projections.head(3)

,Nickname,Opp,Pts,Salary,Pts Per 1k
0,Andrew Luck,vs MIA,21.433333,8400,2.551587
1,Jameis Winston,vs SF,18.266667,7500,2.435556
2,Cam Newton,vs SEA,19.266667,8600,2.240310


___
#### Bringing in the player list from FanDuel:

In [47]:
# Make sure you're selecting the correct file:
player_list=pd.read_csv('player-lists/Week-12/FanDuel-NFL-2018-11-25-30247-players-list.csv')
player_list.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,30247-64401,RB,Saquon,Saquon Barkley,Barkley,22.080000,10,9000,NYG@PHI,NYG,PHI,NaN,NaN,NaN,NaN,NaN
1,30247-28900,RB,Melvin,Melvin Gordon III,Gordon III,23.322222,9,8900,ARI@LAC,LAC,ARI,Q,Knee,NaN,NaN,NaN
2,30247-14185,QB,Cam,Cam Newton,Newton,23.512000,10,8600,SEA@CAR,CAR,SEA,NaN,NaN,NaN,NaN,NaN


> Merging those 2 and making the swap:

In [48]:
master_projections = pd.merge(left=player_list, right=total_projections, how='inner', on='Nickname', suffixes=['_FD', '_tp'])
master_projections['FPPG'] = master_projections['Pts Per 1k'] # Experiment with Pts OR Pts Per 1k
master_projections.drop(['Opp', 'Pts', 'Salary_tp', 'Pts Per 1k'],axis=1, inplace=True)
master_projections.rename(columns={'Salary_FD':'Salary'}, inplace=True)
master_projections.to_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))
master_projections.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,30247-64401,RB,Saquon,Saquon Barkley,Barkley,1.940741,10,9000,NYG@PHI,NYG,PHI,NaN,NaN,NaN,NaN,NaN
1,30247-28900,RB,Melvin,Melvin Gordon III,Gordon III,1.767790,9,8900,ARI@LAC,LAC,ARI,Q,Knee,NaN,NaN,NaN
2,30247-14185,QB,Cam,Cam Newton,Newton,2.240310,10,8600,SEA@CAR,CAR,SEA,NaN,NaN,NaN,NaN,NaN


In [49]:
master_projections.loc[master_projections.Position == 'D', :].sort_values('FPPG', ascending= False)[:5]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
103,30247-12555,D,Baltimore,Baltimore Ravens,Ravens,2.706667,10,5000,OAK@BAL,BAL,OAK,NaN,NaN,NaN,NaN,NaN
117,30247-12554,D,Jacksonville,Jacksonville Jaguars,Jaguars,2.569444,10,4800,JAC@BUF,JAC,BUF,NaN,NaN,NaN,NaN,NaN
163,30247-12545,D,Philadelphia,Philadelphia Eagles,Eagles,2.473684,10,3800,NYG@PHI,PHI,NYG,NaN,NaN,NaN,NaN,NaN
157,30247-12541,D,New England,New England Patriots,Patriots,2.382114,10,4100,NE@NYJ,NE,NYJ,NaN,NaN,NaN,NaN,NaN
169,30247-12529,D,Cleveland,Cleveland Browns,Browns,2.376344,10,3100,CLE@CIN,CLE,CIN,NaN,NaN,NaN,NaN,NaN


In [50]:
master_projections.loc[(master_projections.Nickname == 'Odell Beckham Jr.'), :]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
4,30247-31360,WR,Odell,Odell Beckham Jr.,Beckham Jr.,1.623529,10,8500,NYG@PHI,NYG,PHI,NaN,NaN,NaN,NaN,NaN


___
# Alright! Now let's just use our optimizer tools from above to generate some better lineups

In [51]:
out_players = list(master_projections.loc[(master_projections['Injury Indicator'] == 'IR') | (master_projections['Injury Indicator'] == 'O'), :]['Nickname'].values)
low_scorers = list(master_projections.loc[master_projections['FPPG'] < 0.80, :]['Nickname'].values)

all_removals = out_players + low_scorers
all_removals = list(set(all_removals))

In [52]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
# No stacking on this one

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
# optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=2, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   
 4. WR    Mike Evans                    WR    TB             1.696   7900.0$   
 5. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Gus Edwards                   RB    BAL            1.723   5300.0$   

Fantasy Points 17.937
Salary 59900.0 

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    Nick Chubb                    RB    CLE            1.862   7500.0$   
 3. RB    Joe Mixon                     RB    CIN            1.84    7300.0$   


In [61]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-nostacks.csv'.format(current_week))

In [91]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'WR'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=5, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   
 4. WR    T.Y. Hilton                   WR    IND            1.671   7500.0$   
 5. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Gus Edwards                   RB    BAL            1.723   5300.0$   

Fantasy Points 17.912
Salary 59500.0 

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   


In [63]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-qbwr.csv'.format(current_week))

In [64]:
# Doing the second half with a RB + D stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['RB', 'D'])

    
# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.25):
    print(lineup, '\n')

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   
 4. WR    Mike Evans                    WR    TB             1.696   7900.0$   
 5. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Gus Edwards                   RB    BAL            1.723   5300.0$   

Fantasy Points 17.937
Salary 59900.0 

 1. QB    Andrew Luck                   QB    IND            2.552   8400.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   


 1. QB    Lamar Jackson                 QB    BAL            2.311   7400.0$   
 2. RB    Saquon Barkley                RB    NYG            1.941   9000.0$   
 3. RB    Marlon Mack                   RB    IND            1.805   7000.0$   
 4. WR    Quincy Enunwa                 WR    NYJ            1.44    5000.0$   
 5. WR    Keenan Allen                  WR    LAC            1.671   7700.0$   
 6. WR    Kenny Stills                  WR    MIA            1.447   5300.0$   
 7. TE    Eric Ebron                    TE    IND            1.406   5500.0$   
 8. D     Jacksonville Jaguars          D     JAC            2.569   4800.0$   
 9. FLEX  Leonard Fournette             RB    JAC            1.568   8100.0$   

Fantasy Points 16.157
Salary 59800.0 

 1. QB    Ryan Tannehill                QB    MIA            2.197   6600.0$   
 2. RB    Saquon Barkley                RB    NYG            1.941   9000.0$   
 3. RB    Christian McCaffrey           RB    CAR            1.905   8400.0$   


In [65]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-rbdst.csv'.format(current_week))

In [92]:
# QB + TE stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_FD_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'TE'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
# optimizer.set_min_salary_cap(57000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=5, max_exposure=.35):
    print(lineup, '\n')

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Joe Mixon                     RB    CIN            1.84    7300.0$   
 4. WR    Antonio Brown                 WR    PIT            1.855   8500.0$   
 5. WR    Tyler Boyd                    WR    CIN            1.593   6800.0$   
 6. WR    Jarvis Landry                 WR    CLE            1.683   6100.0$   
 7. TE    Cameron Brate                 TE    TB             1.765   4400.0$   
 8. D     Baltimore Ravens              D     BAL            2.707   5000.0$   
 9. FLEX  Matt Breida                   RB    SF             1.948   6400.0$   

Fantasy Points 17.835
Salary 59900.0 

 1. QB    Jameis Winston                QB    TB             2.436   7500.0$   
 2. RB    James Conner                  RB    PIT            2.008   7900.0$   
 3. RB    Matt Breida                   RB    SF             1.948   6400.0$   


In [67]:
exporter = CSVLineupExporter(optimizer.optimize(n=20, max_exposure=.25))
exporter.export('submissions/Week-{}/fanduel-lineups-qbte.csv'.format(current_week))